# Frame in Square
This project aims is to frame the provided images in a white square of provided size.

In [ ]:
import os
import glob
from PIL import Image

In [ ]:
INPUT_DIR_PATH = "/host/workdir/photos_size"
OUTPUT_DIR_PATH = "/host/workdir/bordure"
print(f"Working on host {os.environ['WORKDIR_PHOTOS']}")

In [ ]:
les_input_images = [path for path in glob.glob(os.path.join(INPUT_DIR_PATH, "*/*.jpg")) if "bordure" not in path and "light" not in path]

print(f"Found {len(les_input_images)} images.")
print("\n".join(
    les_input_images[0:len(les_input_images)//5] 
    + ["..."])
     )

In [ ]:
def generate_output_path(input_path, suffix="None"):
    name = os.path.basename(input_path)
    
    if suffix is not None:
        name = ".".join(name.split(".")[:-1]) + "_" + str(suffix) + "." + name.split(".")[-1]
        
    return os.path.join(OUTPUT_DIR_PATH, name) 

In [ ]:
def export_in_frame(input_image_path, output_image_path, frame_width=2000, propose_4x5=True):
    if os.path.isfile(output_image_path):
        print(f"{output_image_path} already exists, passing")
        return 0
    # Load image
    image = Image.open(input_image_path)
    
    # Get size 
    width, height = image.size
    background_color = (255, 255, 255)
    # background_color = (0, 0, 0)
    new_image = None
    new_image_4x5 = None
    
    # If already square
    if width == height:
        new_image = image
        
    elif width > height:
        new_image = Image.new("RGB", (width, width), background_color)
        new_image.paste(image, (0, (width - height) // 2))
        
    else:
        new_image = Image.new("RGB", (height, height), background_color)
        new_image.paste(image, ((height - width) // 2, 0))
    
    if propose_4x5 and height/width >= 5.0/4 - 0.05:
        print(f"Proposing 4x5 framing")
        # Let's also propose 4x5 framing
        new_image_4x5 = Image.new("RGB", (int(height*(4.0/5)), height), background_color)
        new_image_4x5.paste(image, (int(height*(4.0/5) - width) // 2, 0))
            
    
    if frame_width is not None:
        new_image = new_image.resize((frame_width, frame_width))
        if new_image_4x5 is not None:
            new_image_4x5 = new_image_4x5.resize((frame_width, int(frame_width*(5.0/4))))
        
    
    new_image.save(output_image_path, quality=100)
    if new_image_4x5 is not None:
        new_image_4x5.save(generate_output_path(input_image_path, suffix="4x5"), quality=100)

In [ ]:
les_name = []
for index, input_image_path in enumerate(les_input_images):
    # Check if name already in use
    name = os.path.basename(input_image_path)
    # Count n occurences to define suffix
    if les_name.count(name) > 0:
        suffix = les_name.count(name)
    else:
        suffix = None
    # Add name to memory
    les_name.append(name)
    output_image_path = generate_output_path(input_image_path, suffix=suffix)
        
    
    print(f"{index+1}/{len(les_input_images)}: {input_image_path} > {os.path.basename(output_image_path)}" + " "*30)
    export_in_frame(input_image_path, output_image_path, frame_width=2000, propose_4x5=True)